In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

from pathlib import Path
from hydra import initialize_config_dir, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf
import tqdm
import torch
import matplotlib.pyplot as plt

from tbfm import film
from tbfm import multisession

DATA_DIR = "/home/mmattb/Projects/opto-coproc/data"
sys.path.append(DATA_DIR)
# imported from JNE project
import dataset
meta = dataset.load_meta(DATA_DIR)

OUT_DIR = "data"  # Local data cache; i.e. not reading from the opto-coproc folder.
EMBEDDING_REST_SUBDIR = "embedding_rest"

conf_dir = Path("./conf").resolve()

# Initialize Hydra with the configuration directory
with initialize_config_dir(config_dir=str(conf_dir), version_base=None):
    # Compose the configuration
    cfg = compose(config_name="config")   # i.e. conf/config.yaml

DEVICE = cfg.device
WINDOW_SIZE = cfg.data.trial_len
NUM_HELD_OUT_SESSIONS = cfg.training.num_held_out_sessions

In [ ]:
# Okay, now a stim data loader...
d, paths_held_out = multisession.load_stim_batched(                                                             
    batch_size=10000,                                                               
    window_size=WINDOW_SIZE,                                                               
    session_subdir="torchraw",                                                     
    data_dir=DATA_DIR,                             
    num_held_out_sessions=NUM_HELD_OUT_SESSIONS,                                                      
)
data_train, data_test = d.train_test_split(5000)

held_in_session_ids = next(iter(data_train))[0]

In [8]:
# Gather cached rest embeddings...
embeddings_rest = multisession.load_rest_embeddings(held_in_session_ids, device=DEVICE)

In [ ]:
# Customize cfg here...
cfg.tbfm.module.use_film_bases = True
cfg.ae.training.coadapt = True
# embeddings_rest = None

# Now run...
ms = multisession.build_from_cfg(cfg, train_datas, device=DEVICE)
model_optims = multisession.get_optims(cfg, ms)
embeddings_stim, results = multisession.train_from_cfg(cfg, ms, data_train, model_optims, embeddings_rest, data_test=data_test, epochs=7000)